# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path = "../output_data/cities.csv"
weather_df = pd.read_csv(path)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Upernavik,72.79,-56.15,34.50,87,0,7.47,GL,1592439186
1,Hermanus,-34.42,19.23,48.99,84,21,1.01,ZA,1592439186
2,Punta Arenas,-53.15,-70.92,28.40,92,0,5.57,CL,1592438936
3,Ahipara,-35.17,173.17,60.96,80,63,8.70,NZ,1592439187
4,Trairi,-3.28,-39.27,75.61,90,13,8.43,BR,1592439187


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrowing down the DataFrame with ideal vacation conditions
narrow_df=weather_df.loc[(weather_df["Max Temp"]<80) & (weather_df["Cloudiness"]==0)]
narrow2_df=narrow_df.loc[(narrow_df["Max Temp"]>70) & (narrow_df["Wind Speed"]<5)]
narrow2_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 37 to 491
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        10 non-null     object 
 1   Lat         10 non-null     float64
 2   Lng         10 non-null     float64
 3   Max Temp    10 non-null     float64
 4   Humidity    10 non-null     int64  
 5   Cloudiness  10 non-null     int64  
 6   Wind Speed  10 non-null     float64
 7   Country     9 non-null      object 
 8   Date        10 non-null     int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 800.0+ bytes


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=narrow2_df[["City","Country","Lat", "Lng",]].copy()
hotel_df["Hotel Name"]=""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
37,Lobito,AO,-12.35,13.55,
39,Mossamedes,AO,-15.20,12.15,
141,Pontes e Lacerda,BR,-15.23,-59.34,
249,Guozhen,CN,34.37,107.36,
251,Goderich,CA,43.75,-81.72,


In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    #print(json.dumps(hotel_name, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [8]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
37,Lobito,AO,-12.35,13.55,Casa Rosa Hotel Residence
39,Mossamedes,AO,-15.20,12.15,Hotel ibis Styles Iu Namibe
141,Pontes e Lacerda,BR,-15.23,-59.34,Plaza Hotel
249,Guozhen,CN,34.37,107.36,Lily Garden Hotel
251,Goderich,CA,43.75,-81.72,Samuels Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

# Create a marker_layer using the poverty list to fill the info box
hotel_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))